In [25]:
from gensim.models.ldamulticore import LdaMulticore
import multiprocessing as mp
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary

In [26]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("words")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/david/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/david/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [27]:
stop_words_nltk_en = set(stopwords.words("english"))
stop_words_nltk_es = set(stopwords.words("spanish"))

In [28]:
tk = TweetTokenizer()
porter = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [29]:
def textprep(line):
    tokens = tk.tokenize(str(line))
    tokens = [w.lower() for w in tokens if len(w) > 1]
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w for w in tokens if w not in stop_words_nltk_en]
    tokens = [w for w in tokens if w not in stop_words_nltk_es]
    return tokens

In [30]:
path_in = "twitterClimateData.csv"
df = pd.read_csv(path_in, delimiter=";")
df = df[["text", "hashtags", "search_hashtags"]]

In [31]:
topics = df["search_hashtags"].unique()
k = len(topics)
print(topics, k)

['#climatestrike' '#climatechange' '#greennewdeal' '#climatecrisis'
 '#climateaction' '#fridaysforfuture' '#environment' '#globalwarming'
 '#actonclimate' '#sustainability' '#savetheplanet' '#bushfires'] 12


We identify common words in the topics of the tweets, so for that reason we reduce `k` to 11


In [32]:
k -= 1

In [33]:
df["tokens_text"] = df.apply(lambda row: textprep(row["text"]), axis=1)

In [34]:
dictionary = Dictionary(df.tokens_text)
corpus = [dictionary.doc2bow(doc) for doc in df.tokens_text]

In [35]:
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(
    dictionary.doc2bow, [sentence for sentence in df.tokens_text]
)
pool.close()

In [36]:
lda_model = LdaMulticore(
    doc_term_matrix, num_topics=k, id2word=dictionary, passes=10, workers=10
)

In [37]:
def assign_topic(lda_model, dictionary, doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    vector = sorted(vector, key=lambda item: -item[1])
    return vector

In [38]:
df["topics"] = df.apply(lambda row: assign_topic(lda_model, dictionary, row["tokens_text"]), axis=1)
df.head()

,text,hashtags,search_hashtags,tokens_text,topics
0,"2020 is the year we #votethemout, the year we ...",#votethemout #climatestrike #rebelforlife,#climatestrike,"[year, year, hearts, year, without, liveable, ...","[(1, 0.93936783)]"
1,Winter has not stopped this group of dedicated...,#climatefriday #climatestrike #ClimateAction,#climatestrike,"[winter, stopped, group, dedicated, climate, a...","[(2, 0.50264126), (1, 0.29887015), (3, 0.11765..."
2,WEEK 55 of #ClimateStrike at the @UN. Next wee...,#ClimateStrike,#climatestrike,"[week, next, week, heads, year, striking, time...","[(3, 0.7509676), (6, 0.21755558)]"
3,"A year of resistance, as youth protests shaped...",#greta #gretathunberg #climatechange #fridaysf...,#climatestrike,"[year, resistance, youth, protests, shaped, cl...","[(8, 0.51335824), (3, 0.41843662)]"
4,HAPPY HOLIDAYS #greta #gretathunberg #climate...,#greta #gretathunberg #climatechange #fridaysf...,#climatestrike,"[happy, holidays, energy, house, team]","[(2, 0.8484722), (4, 0.015153016), (7, 0.01515..."


In [39]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))

# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=3))

[(1, 0.93936884)]
[(10, '0.007*"new" + 0.007*"time" + 0.005*"real"'), (4, '0.033*"climate" + 0.017*"change" + 0.016*"action"'), (8, '0.027*"climate" + 0.021*"change" + 0.012*"today"'), (0, '0.022*"city" + 0.020*"today" + 0.016*"hall"'), (3, '0.039*"climate" + 0.016*"change" + 0.012*"action"'), (6, '0.016*"need" + 0.012*"us" + 0.012*"future"'), (5, '0.011*"dc" + 0.007*"see" + 0.006*"make"'), (7, '0.009*"like" + 0.009*"new" + 0.006*"people"'), (1, '0.012*"err" + 0.008*"today" + 0.007*"green"'), (2, '0.012*"world" + 0.010*"house" + 0.010*"young"')]


In [40]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

In [41]:
lda_topic_assignment

[(1, 0.93936867),
 (2, 0.4997787),
 (3, 0.7509112),
 (8, 0.51344365),
 (2, 0.8484722),
 (3, 0.8484736),
 (0, 0.09090909),
 (0, 0.09090909),
 (5, 0.68150675),
 (6, 0.8862444),
 (3, 0.87011933),
 (9, 0.96362203),
 (0, 0.09090909),
 (9, 0.5695433),
 (2, 0.4776912),
 (5, 0.9636237),
 (7, 0.46931073),
 (2, 0.54710215),
 (3, 0.47057638),
 (0, 0.9393668),
 (4, 0.53728586),
 (6, 0.9173361),
 (2, 0.81170106),
 (0, 0.09090909),
 (0, 0.09090909),
 (1, 0.84839594),
 (8, 0.9567011),
 (3, 0.7726297),
 (3, 0.9350306),
 (8, 0.93002784),
 (0, 0.59602255),
 (8, 0.9464833),
 (1, 0.49350035),
 (5, 0.73142934),
 (3, 0.5193656),
 (3, 0.8863377),
 (2, 0.8863301),
 (0, 0.09090909),
 (5, 0.93004525),
 (5, 0.5588453),
 (5, 0.6375691),
 (4, 0.53328764),
 (0, 0.9242159),
 (0, 0.8989688),
 (9, 0.87007606),
 (4, 0.76456034),
 (2, 0.81817317),
 (6, 0.9173335),
 (4, 0.95452964),
 (8, 0.87400347),
 (9, 0.8701101),
 (8, 0.91732484),
 (9, 0.8447565),
 (4, 0.9090486),
 (6, 0.94948214),
 (3, 0.66604376),
 (2, 0.77271795),

In [42]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.261854  0.043853       1        1   8.115745
1      0.027362 -0.157799       2        1   7.167304
2     -0.010748  0.048628       3        1   9.934688
3     -0.040032  0.102424       4        1  11.686811
4     -0.078666  0.084894       5        1  10.599122
5      0.003094 -0.023307       6        1   7.194560
6     -0.068251  0.047275       7        1  10.264283
7     -0.045671 -0.089103       8        1  10.658796
8     -0.001840  0.055713       9        1   9.676406
9     -0.012826 -0.018236      10        1   6.889003
10    -0.034277 -0.094341      11        1   7.813281, topic_info=           Term         Freq        Total Category  logprob  loglift
12      climate  9201.000000  9201.000000  Default  30.0000  30.0000
211        city  1588.000000  1588.000000  Default  29.0000  29.0000
77      capitol  1491.000000  1491.000000  Default  28.0000  28.0000
215        hall  1235.000000  1235.000000  Default  27.0000  27.0000
41       change  5038.000000  5038.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1455  solutions   135.605030   718.269497  Topic11  -5.9890   0.8822
366        know   142.802426  1181.052651  Topic11  -5.9373   0.4366
700        plan   135.516930   734.105152  Topic11  -5.9897   0.8598
256        like   145.941830  2675.148379  Topic11  -5.9155  -0.3592
120         let   136.106868  1119.709441  Topic11  -5.9853   0.4420

[912 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
15910      9  0.985582      aarey
3463       7  0.907879    abandon
5555       5  0.964549        abc
4991       2  0.022374  according
4991       3  0.052206  according
...      ...       ...        ...
14591      8  0.024844         एक
14341      2  0.038705         और
14341      6  0.058057         और
14341      8  0.890208         और
14450      8  0.978950         पर

[3367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])